# Asset-Liability Management

Funding ratio is an important indicator for telling us what is the fraction of 
liabilities that you are able to cover given the existing assets.

If the funding ratio is 100% or above that you are in a good shape because you are able to cover those liabilities.




# Present Value of Liabilities and Funding Ratio

In this lab session, we'll examine how to discount future liabilities to compute the present value of future liabilities, and measure the funding ratio.

The funding ratio is the ratio of the current value of assets to the present value of the liabilities.

In order to compute the present value, we need to discount the amount of the liability based on the relevant interest rate derived from the yield curve.

For simplicity, we'll assume that the yield curve is flat, and so the interest rate is the same for all horizons.

The present value of a set of liabilities $L$ where each liability $L_i$ is due at time $t_i$ is give by:

$$ PV(L) = \sum_{i=1}^{k} B(t_i) L_i$$

where $B(t_i)$ is the price of a pure discount bond that pays 1 dollar at time $t_i$

If we assume the yield curve is flat and the annual rate of interest is $r$ then $B(t)$ is given by

$$B(t) = \frac{1}{(1+r)^t}$$



In [2]:
import numpy as np
import pandas as pd
import risk_kit as rk

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays a dollar at a time t given interest rate r
    """
    return (1+r)**(-t)

In [4]:
discount(10, 0.03)

0.7440939148967249

In [5]:
0.7440939148967249*(1.03**10)

1.0

In [8]:
def pv(l ,r):
    """
    Computes the peresent value of a sequence of liabilities
    l is indexed by the time and the values are the amounts of each liability
    returns the peresent value of the sequence
    """

    dates = l.index
    discounts = discount(dates, r)
    return (discounts*l).sum()

In [7]:
liabilities = pd.Series(data=[1, 1.5, 2, 2.5], index = [3, 3.5, 4, 4.5])

In [10]:
pv(liabilities, 0.03)

6.233320315080045

In [11]:
liabilities

3.0    1.0
3.5    1.5
4.0    2.0
4.5    2.5
dtype: float64

In [13]:
def funding_ratio(asset_value, liabilities, r):
    """
    Computes the funding ratio of some assets given liabilities and interest rate
    """
    return asset_value/pv(liabilities, r)

In [15]:
funding_ratio(5, liabilities, 0.03)
# Means that we have 80% of our liabilities covered.

0.8021407126958777

In [17]:
funding_ratio(5, liabilities, 0.02)
# Interest rate going down for funding ratios is not good as our 
# bonds will not grow as fast as if there was more interest rates

0.7720304366941648

In [18]:
funding_ratio(5, liabilities, 0.05)

0.8649082768407927

In [19]:
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [20]:
def show_funding_ratio(assets, r):
    fr = funding_ratio(assets, liabilities, r)
    print(f'{fr*100:.2f}')

In [22]:
controls = widgets.interactive(show_funding_ratio, assets = widgets.IntSlider(min=1, max=10, step=1, value=5),
                               r = (0, 0.20, 0.01)
                               )

display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…